At the end to run the output plot in the cool interactive way, just run  serve --show Untitled.ipynb from the same die as this notebook

In [1]:
import geopandas as gpd
import pandas as pd
import json


In [2]:
# %%writeandexecute -i identifier load.py

def loadGeopandas():    
    shapefile = 'data/countries_110m/ne_110m_admin_0_countries.shp'
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
    gdf.head()
    gdf.columns = ['country', 'country_code', 'geometry']
    gdf.head()
    print(gdf[gdf['country'] == 'Antarctica'])
    #Drop row corresponding to 'Antarctica'
    gdf = gdf.drop(gdf.index[159])
    return gdf

In [3]:
gdf = loadGeopandas()

        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  


In [2]:
def getTimeSeries(fname):
    print('Rading from', fname)
    df = pd.read_csv(fname)
    df.head()
    countryData = df.groupby('Country/Region').sum()
    countryData = countryData.drop(['Lat', 'Long'], axis=1).T
    return countryData.set_index(pd.DatetimeIndex(countryData.index))

def fetchCountryData(freq='1d'):
    gisandata = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_'
    
    confirmed = getTimeSeries(f'{gisandata}19-covid-Confirmed.csv ')
    dead = getTimeSeries(f'{gisandata}confirmed_deaths.csv')
    recovered = getTimeSeries(f'{gisandata}confirmed_recovered.csv')
    
    data = pd.DataFrame([confirmed, dead, recovered],
                ['confirmed', 'dead', 'recovered']).T
    
    data = data[data.confirmed>0].resample(freq).max() # start with first confirmed case
    data[['new_confirmed', 'new_dead', 'new_recovered']] = data.diff()
    
    data['mortality'] = data.dead / data.confirmed * 100
    
    return data

In [3]:
df = fetchCountryData()
df.info()

NameError: name 'pd' is not defined

In [ ]:
df['code'].isnull()

In [ ]:
#Filter data for year 2016.
df_2016 = df[df['year'] == 2016]
#Perform left merge to preserve every row in gdf.
merged = gdf.merge(df_2016, left_on = 'country_code', right_on = 'code', how = 'left')
#Replace NaN values to string 'No data'.
# merged.fillna('No data', inplace = True)

In [ ]:
#Read data to json.
merged_json = json.loads(merged.to_json())
#Convert to String like object.
json_data = json.dumps(merged_json)

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

from bokeh.io import curdoc
from bokeh.models import Slider, HoverTool
from bokeh.layouts import widgetbox, row, column

#Input GeoJSON source that contains features for plotting.

def json_data(selectedYear):
    yr = selectedYear
    df_yr = df[df['year'] == yr]
    merged = gdf.merge(df_yr, left_on = 'country_code', right_on =     'code', how = 'left')
#     merged.fillna('No data', inplace = True)
    merged_json = json.loads(merged.to_json())
    json_data = json.dumps(merged_json)
    return json_data

geosource = GeoJSONDataSource(geojson = json_data(2016))
#Define a sequential multi-hue color palette.
palette = brewer['YlGnBu'][8]
#Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]
#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 40, nan_color = '#d9d9d9')
#Define custom tick labels for color bar.
tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%', '25':'25%', '30':'30%','35':'35%', '40': '>40%'}
#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)
#Create figure object.
p = figure(title = 'Share of adults who are obese, 2016', plot_height = 600 , plot_width = 950, toolbar_location = None)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p.patches('xs','ys', source = geosource,fill_color = {'field' :'per_cent_obesity', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
#Specify figure layout.
p.add_layout(color_bar, 'below')

# Define the callback function: update_plot
def update_plot(attr, old, new):
    yr = slider.value
    new_data = json_data(yr)
    geosource.geojson = new_data
    p.title.text = 'Share of adults who are obese, %d' %yr
    
# Make a slider object: slider 
slider = Slider(title = 'Year',start = 1975, end = 2020, step = 1, value = 2020)
slider.on_change('value', update_plot)
# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(p,widgetbox(slider))
curdoc().add_root(layout)


#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
show(p)